# Topic Modeling on News Articles

Topic Modelling to segregate news report data to different topics using Gensim, NLTK, Spacy.

Topic modelling as the name suggests, it is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus.

Topics can be defined as “a repeating pattern of co-occurring terms in a corpus”. A good topic model should result in <li>– “health”, “doctor”, “patient”, “hospital” for a topic – Healthcare, <li>and “farm”, “crops”, “wheat” for a topic – “Farming”. 

We have a dataset which consists of **News articles** and our task is to assign topics to those articles.
    
We will do a simple LSI and a LDA method to figure out the topics

Task consist of :

<lu> **1. Loading the data**
    
<lu> **2. Clean the Data**
Transforming text into something an algorithm can digest it a complicated process. We cannot feed the data as it is, some preprocessing needs to be done. In this task we will be doing some preprocessing to convert our data in a form that we can feed our model with.
    
<li> **. Handling the Stop-words**
    
Text may contain stop words like ‘the’, ‘is’, ‘are’. Stop words can be filtered from the text to be processed. There is no universal list of stop words in nlp research, however the nltk module contains a list of stop words. We will remove these stopwords in this task.
    
<li> **. Lemmatization**
    
<lu> **3. TF-IDF Vectorization**
    
Apart from Count vectorizer an alternative to calculate word frequencies , and by far the most popular method is called TF-IDF. This is an acronym than stands for “Term Frequency – Inverse Document” Frequency which are the components of the resulting scores assigned to each word.
    
   
<lu> **4. Topic Modelling using Gensim's LDA**
    
LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 
    
<lu> **4. Topic modelling using LSA**
    
Latent Semantic Analysis, or LSA, is one of the foundational techniques in topic modeling. The core idea is to take a matrix of what we have — documents and terms — and decompose it into a separate document-topic matrix and a topic-term matrix.

# Loading the data

In [1]:
#imports
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
from collections import Counter
import operator
import numpy as np




In [2]:
import warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now"

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# Dictionary with 'data' as key and each review as an element of list
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
dataset

In [ ]:
# list of articles
documents = dataset.data

In [ ]:
documents

In [ ]:
# Dataframe with a column `document` containing all the articles
news_df = pd.DataFrame({'document':documents})

In [ ]:
news_df

In [ ]:
print(news_df.head())

# Clean the Data

Transforming text into something an algorithm can digest it a complicated process. We cannot feed the data 
as it is, some preprocessing needs to be done. 
In this task we will be doing some preprocessing to convert our data in a form that we can feed our model with.'''



In [ ]:
news_df = news_df[:100]
tokenized_doc = news_df['document'].str.replace("[^a-zA-Z#]", " ")
news_df.shape

In [ ]:
def clean_doc(text):
    #for token in text:
    text =  ' '.join([w.lower() for w in text.split() if len(w)>3])
    return text;

tokenized_doc = tokenized_doc.apply(clean_doc)
tokenized_doc

#### Handling the Stop-words and Punctuation-->Lemmatization

In [ ]:
#import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
  
stopwords_set = set(stopwords.words('english'))
custom = list(stopwords_set)+list(punctuation)

def stopWordRemoval(text):
    text = word_tokenize(text)
    text = ' '.join([word for word in text if word not in custom])
    return text

tokenized_doc = tokenized_doc.apply(stopWordRemoval)
tokenized_doc

# Initializing wordnet lemmatizer
lemmatizer = WordNetLemmatizer()

def lemData(text):
    text = word_tokenize(text)
    newText = []
    for word in text:
        newText.append(lemmatizer.lemmatize(word))
    return ' '.join(newText)

doc_clean = tokenized_doc.apply(lemData)
doc_clean


In [ ]:
# Creating a list of documents from the complaints column
list_of_docs = doc_clean.tolist()

# Implementing the function for all the complaints of list_of_docs
doc_clean = [doc.split() for doc in list_of_docs]

In [ ]:
doc_clean

## LDA ##



Some of the initialisations necessary for LDA are:
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **word2id** is a mapping from words (strings) to word ids (integers). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **vocab** is a list of words (strings). It is used to determine the various words used in the text, as well as for debugging and topic printing.

In [ ]:
import gensim


from gensim import corpora
import string
from gensim.models.ldamodel import LdaModel

# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary = corpora.Dictionary(doc_clean)

# Creating the corpus
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the LDA model
ldamodel = LdaModel(corpus=doc_term_matrix, num_topics=5,id2word=dictionary, random_state=20, passes=30)

In [ ]:
from pprint import pprint
# printing the topics
pprint(ldamodel.print_topics())

## LDA on TFIDF vectoriser

In [ ]:
from gensim import models
# train the model
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [ ]:
# Creating the LDA model
ldamodel = LdaModel(corpus=corpus_tfidf, num_topics=5,id2word=dictionary, random_state=20, passes=30)

# printing the topics
pprint(ldamodel.print_topics())

# Evaluation Metric

#### What is topic coherence?

Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference.

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Coherence Score
#Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference.
coherence_model_lda = CoherenceModel(model=ldamodel, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Best K topics

Let’s now define a function to compute the topic coherence for a given num_topics value and apply it to the num_topics values used in Latent Semantic Indexing in Python

The last step is to find the optimal number of topics
We need to build many LDA models with different values of the number of topics (k) and pick the one that gives the highest coherence value.



In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, step, start=2):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
#To DO 
#plot coherence distribution and choose best k
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=doc_clean, start=2, limit=10, step=1)


In [ ]:
coherence_values

In [ ]:
# Show graph
import matplotlib.pyplot as plt
limit=10; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

<li> Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics. 

<li>If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The above plot shows that coherence score increases with the number of topics, with a decline between 3 to 4.

Now, choosing the number of topics still depends on your requirement because topic around 5 have good coherence scores but may have repeated keywords in the topic. Topic coherence gives you a good picture so that you can take better decision.

# pyLDAvis Visualization

pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

In [ ]:
#!pip install pyLDAvis.gensim


import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()#enable automatic D3 display of prepared model data in the IPython notebook.

vis = gensimvis.prepare(ldamodel, doc_term_matrix, dictionary)
vis

 ## How to predict the topics for a new piece of text?

In [ ]:
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True, random_state=1, remove=('headers', 'footers', 'quotes'))
num = 100
unseen_document = newsgroups_test.data[num]
print(unseen_document)

Assuming that you have already built the topic model, you need to take the text through the same routine of transformations and before predicting the topic.

For our case, the order of transformations is:

sent_to_words() –> lemmatization() –> vectorizer.transform() –> best_lda_model.transform()

You need to apply these transformations in the same order. So to simplify it, let’s combine these steps into a preprocess() function.

In [ ]:
# Define function to predict topic for a given text document.


def preprocess(text):
    
    # Step 1: Clean with simple_preprocess
    mytext_2 =  text.replace("[^a-zA-Z#]", " ")
    mytext_3=clean_doc(mytext_2)
    mytext_4=stopWordRemoval(mytext_3)


    # Step 2: Lemmatize
    mytext_5 = lemData(mytext_4)
    
    # Step 3: Vectorize transform
   # mytext_6 = dictionary.doc2bow(mytext_5)#tfidf.transform(mytext_3)

    # Step 4: LDA Transform
    #topic_probability_scores = ldamodel.transform(mytext_6)
    #topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return mytext_5.split()#topic, topic_probability_scores

# Predict the topic

#topic, prob_scores = predict_topic(text = unseen_document)
#print(topic)

In [ ]:
preprocess(unseen_document)

In [ ]:
#  predicting new text 
new_text_corpus =  dictionary.doc2bow(preprocess(unseen_document))
ldamodel[new_text_corpus]

In [ ]:
for index, score in sorted(ldamodel[new_text_corpus], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 5)))

# Gensim LSI 

In [ ]:
from gensim.models.lsimodel import LsiModel

In [ ]:
# Creating the LSi model
lsimodel = LsiModel(corpus=doc_term_matrix, num_topics=5, id2word=dictionary)
pprint(lsimodel.print_topics())

# Word Cloud Visualization

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stopwords,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = ldamodel.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
fig.savefig('word_cloud.png')
plt.show()